# Creating a pandas dataframe from the Wikipedia page

In [1]:
# Imported Libraries

!pip install folium
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("Libaries imported")

     |████████████████████████████████| 92kB 15.2MB/s eta 0:00:01
Libaries imported


# Scraping the data from the Wikipedia webpage into a pandas Dataframe

In [2]:
tordata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Parsing the html data into a BeautifulSoup object

In [4]:
soup = BeautifulSoup(tordata, 'html.parser')

#Creating 3 lists to store the data

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

#Created Dataframe, from the three lists

In [9]:
soup.find('table').find_all('tr')
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\\n')) 

In [11]:
soup
df_toronto = pd.DataFrame({"PostalCode":postalCodeList, "Borough":boroughList, "Neighborhood":neighborhoodList})
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#Cells not assigned Borough dropped

In [12]:
df_torbordrop = df_toronto[df_toronto.Borough != "Not assigned"].reset_index(drop = True)
df_torbordrop.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


#Neighborhoods with the same Borough grouped together

In [40]:
df_toronto_borgp = df_torbordrop.groupby(["PostalCode", "Borough"], as_index = False).agg(lambda x:",".join(x))
df_toronto_borgp.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n,Malvern\n,Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n,Hig..."
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n,Guildwoo..."
3,M1G,Scarborough,"Woburn\n,Woburn\n"
4,M1H,Scarborough,"Cedarbrae\n,Cedarbrae\n"


#Replacing Neighborhoods with 'Not assigned' as equal to value of Borough

In [14]:
df_toronto_borgp['Neighborhood'].replace("Not assigned", df_toronto_borgp['Borough'], inplace = True)
df_toronto_borgp.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n,Malvern\n,Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n,Hig..."
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n,Guildwoo..."
3,M1G,Scarborough,"Woburn\n,Woburn\n"
4,M1H,Scarborough,"Cedarbrae\n,Cedarbrae\n"


#Using the .shape to print the number of rows in the dataframe

In [15]:
print("The number of rows in the data frame:", df_toronto_borgp.shape)

The number of rows in the data frame: (103, 3)


# Obtaining geospatial data after importing the csv file

In [17]:
geosp_df = pd.read_csv('http://cocl.us/Geospatial_data')
geosp_df.head(12)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


# Exploring and clustering neighborhoods in Toronto

#Merging the two dataframes

In [18]:
df_toronto_borgp = pd.merge(df_toronto_borgp, geosp_df, how = 'left', left_on = 'PostalCode', right_on = 'Postal Code', validate = "1:1")
df_toronto_borgp.drop(labels = 'Postal Code', axis = 1, inplace = True)
df_toronto_borgp.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n,Rouge\n,Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n,Hig...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n,Guildwoo...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn\n,Woburn\n",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae\n,Cedarbrae\n",43.773136,-79.239476


#The new dataframe

In [19]:
df_toronto_borgp.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n,Rouge\n,Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n,Hig...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n,Guildwoo...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn\n,Woburn\n",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae\n,Cedarbrae\n",43.773136,-79.239476


In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.
      format(len(df_toronto_borgp['Borough'].unique()),
       df_toronto_borgp.shape[0]
    )
)                                                                   

The dataframe has 11 boroughs and 103 neighborhoods.


#Creating maps

In [21]:
# Finding the coordinates of Toronto

from geopy.geocoders import Nominatim
address = 'Toronto'
geolocator = Nominatim(user_agent = "my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {},{}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963,-79.387207.


In [22]:
# The map of Toronto

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)
for lat, lng, borough, neighborhood in zip(df_toronto_borgp['Latitude'], df_toronto_borgp['Longitude'], df_toronto_borgp['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
       [lat, lng],
       radius = 5,
       popup = label,
       color = 'blue',
       fill = True,
       fill_color = '#3186cc',
       fill_opacity = 0.7,
       parse_html = False).add_to(map_toronto)
    
map_toronto

#Considering a much smaller town, East York

In [23]:
eastyork_data = df_toronto_borgp[df_toronto_borgp['Borough'] == 'East York'].reset_index(drop = True)
eastyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Woodbine Gardens\n,Parkview Hill\n,Woodbine Ga...",43.706397,-79.309937
1,M4C,East York,"Woodbine Heights\n,Woodbine Heights\n",43.695344,-79.318389
2,M4G,East York,"Leaside\n,Leaside\n",43.709060,-79.363452
3,M4H,East York,"Thorncliffe Park\n,Thorncliffe Park\n",43.705369,-79.349372
4,M4J,East York,"East Toronto\n,East Toronto\n",43.685347,-79.338106


In [24]:
address = 'East York'
geolocator = Nominatim(user_agent = "my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of East York are {},{}.'.format(latitude, longitude))

The geographical coordinates of East York are 43.6913391,-79.3278212.


In [25]:
map_eastyork = folium.Map(location = [latitude, longitude], zoom_start = 11)
for lat, lng, borough, neighborhood, PostalCode in zip(eastyork_data['Latitude'], eastyork_data['Longitude'], eastyork_data['Borough'], eastyork_data['Neighborhood'], eastyork_data['PostalCode']):
    label = '{}, {}, {}'.format(borough, neighborhood, PostalCode)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
      [lat, lng],
      radius = 5,
      popup = label,
      color = 'blue',
      fill = True,
      fill_color = '#3186cc',
      fill_opacity = 0.7,
      parse_html = False).add_to(map_eastyork)
    
map_eastyork

#Using FOURSQUARE

In [26]:
# FOURSQUARE credentials and version
CLIENT_ID = 'L351DHR2FSTSDETU0K5X3IZMC5XCTH5GJMOTYKAE0BEVPVO3'
CLIENT_SECRET = 'U0NV33KA4DBR4L5ZDYKM2XXXOO4RFH5DI1XNRFUGJDIF2MVR'
VERSION = '20180605'
print('Your credentials:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID:L351DHR2FSTSDETU0K5X3IZMC5XCTH5GJMOTYKAE0BEVPVO3
CLIENT_SECRET:U0NV33KA4DBR4L5ZDYKM2XXXOO4RFH5DI1XNRFUGJDIF2MVR


#Exploration of the first neighborhood in the dataset

In [27]:
eastyork_data.loc[0, 'Neighborhood']

'Woodbine Gardens\n,Parkview Hill\n,Woodbine Gardens\n,Parkview Hill\n'

#Getting the neighborhood's latitude and longitude

In [28]:
neighborhood_latitude = eastyork_data.loc[0, 'Latitude']
neighborhood_longitude = eastyork_data.loc[0, 'Longitude']

# Neighborhood name
neighborhood_name = eastyork_data.loc[0, 'Neighborhood']

print('Latitude and longitude of {} are {}, {}.'.format(neighborhood_name,
                                                       neighborhood_latitude,
                                                       neighborhood_longitude))

Latitude and longitude of Woodbine Gardens
,Parkview Hill
,Woodbine Gardens
,Parkview Hill
 are 43.7063972, -79.309937.


#Clustering neighborhoods

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500, LIMIT = 100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Append the venues list
        venues_list.append([(
            names,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results]) 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                        'Neighborhood Latitude',
                        'Neighborhood Longitude',
                        'Venue',
                        'Venue Latitude',
                        'Venue Longitude',
                        'Venue Category']
    return(nearby_venues)
eastyork_venues = getNearbyVenues(names = df_toronto_borgp['Neighborhood'], 
                                  latitudes = df_toronto_borgp['Latitude'],
                                  longitudes = df_toronto_borgp['Longitude'])
eastyork_venues.head()

Rouge
,Malvern
,Rouge
,Malvern

Highland Creek
,Rouge Hill
,Port Union
,Highland Creek
,Rouge Hill
,Port Union

Guildwood
,Morningside
,West Hill
,Guildwood
,Morningside
,West Hill

Woburn
,Woburn

Cedarbrae
,Cedarbrae

Scarborough Village
,Scarborough Village

East Birchmount Park
,Ionview
,Kennedy Park
,East Birchmount Park
,Ionview
,Kennedy Park

Clairlea
,Golden Mile
,Oakridge
,Clairlea
,Golden Mile
,Oakridge

Cliffcrest
,Cliffside
,Scarborough Village West
,Cliffcrest
,Cliffside
,Scarborough Village West

Birch Cliff
,Cliffside West
,Birch Cliff
,Cliffside West

Dorset Park
,Scarborough Town Centre
,Wexford Heights
,Dorset Park
,Scarborough Town Centre
,Wexford Heights

Maryvale
,Wexford
,Maryvale
,Wexford

Agincourt
,Agincourt

Clarks Corners
,Sullivan
,Tam O'Shanter
,Clarks Corners
,Sullivan
,Tam O'Shanter

Agincourt North
,L'Amoreaux East
,Milliken
,Steeles East
,Agincourt North
,L'Amoreaux East
,Milliken
,Steeles East

L'Amoreaux West
,L'Amoreaux West

Upper Rouge
,Upper Rouge

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"0 Rouge\n,Malvern\n,Rouge\n...",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"0 Rouge\n,Malvern\n,Rouge\n...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


#The number of rows and columns in (size of) the dataframe

In [30]:
print("The number of rows and columns in the dataframe:", eastyork_venues.shape)

The number of rows and columns in the dataframe: (2244, 7)


#Venues returned for each neighborhood

In [31]:
eastyork_venues.groupby('Neighborhood')

In [39]:
eastyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"0 Rouge\n,Malvern\n,Rouge\n...",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"0 Rouge\n,Malvern\n,Rouge\n...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"0 Rouge\n,Malvern\n,Rouge\n...",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


#Analyzing each Neighborhood

In [33]:
eastyork_onehot = pd.get_dummies(eastyork_venues[['Venue Category']], prefix = "", prefix_sep = "")

# Adding the neighborhood column back to the dataframe
eastyork_onehot['Neighborhood'] = eastyork_venues['Neighborhood']

# Take the Neighborhood column to the first column
fixed_columns = [eastyork_onehot.columns[-1]] + list(eastyork_onehot.columns[:-1])
eastyork_onehot = eastyork_onehot[fixed_columns]
eastyork_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's 

In [34]:
eastyork_onehot.shape

(2244, 280)

#Grouping by Neighborhood 

In [35]:
eastyork_onehot.groupby('Neighborhood')

In [38]:
eastyork_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's 

In [37]:
eastyork_venues.shape

(2244, 7)